In [24]:
#import transformers
#print(transformers.__version__)


In [25]:
#pip uninstall transformers -y

In [26]:
#pip install transformers==4.40.1

In [27]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
import transformers
from transformers import AutoModelForCausalLM

In [ ]:
data= pd.read_csv("data/final_concatenated_data.csv")
data

,Unnamed: 0,date,S199,S200,S332B,S332BN,S332C,S332D,S332DX1,S12A,...,NP205_rain,NP205_stage,P33_rain,P33_stage,NP205_PET,P33_PET,G620_water_level,SWEVER4_stage,TSH_stage,NP62_stage
0,0,10/16/2020,242.17,279.69,218.42,218.42,445.77,481.38,0.0,460,...,2.28,6.925,0.106,7.133,2.09,2.09,5.92,2.92,3.03,4.04
1,1,10/17/2020,296.05,294.85,265.13,265.13,587.35,556.24,0.0,490,...,0.01,7.025,0.000,7.133,3.87,3.87,5.97,2.95,3.08,4.05
2,2,10/18/2020,277.85,294.24,264.23,264.23,514.45,503.14,0.0,472,...,0.00,7.008,0.088,7.138,3.51,3.51,5.98,2.93,3.06,4.05
3,3,10/19/2020,224.13,294.72,264.39,264.39,491.06,537.91,0.0,466,...,0.11,6.986,0.256,7.167,1.98,1.98,5.98,2.91,3.05,4.06
4,4,10/20/2020,260.51,295.06,263.38,263.38,489.25,558.77,0.0,472,...,0.37,6.977,0.678,7.195,1.27,1.27,5.99,2.90,3.06,4.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1406,1406,8/22/2024,297.72,294.23,0.17,0.17,30.02,559.39,0.0,358,...,1.72,7.289,0.050,7.466,2.75,2.75,6.13,2.85,2.78,3.88
1407,1407,8/23/2024,240.47,293.74,0.45,0.45,0.29,491.92,0.0,375,...,0.04,7.360,0.000,7.447,2.75,2.75,6.18,2.90,2.81,3.88
1408,1408,8/24/2024,242.45,293.75,0.00,0.00,0.00,439.40,0.0,377,...,0.00,7.360,0.010,7.435,2.75,2.75,6.20,2.88,2.80,3.88
1409,1409,8/25/2024,286.80,294.24,0.00,0.00,0.00,470.49,0.0,380,...,0.00,7.345,0.270,7.440,2.75,2.75,6.19,2.86,2.80,3.98


In [29]:
# Convert 'date' column to proper datetime format
data['date'] = pd.to_datetime(data['date'])

# Verify the column types to confirm the 'date' is now datetime
print(data.info())

# When displaying or exporting, you can format the date as MM/DD/YYYY
data['formatted_date'] = data['date'].dt.strftime('%m/%d/%Y')

# Print the first few rows to see the formatted date
print(data[['date', 'formatted_date']])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1411 entries, 0 to 1410
Data columns (total 39 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Unnamed: 0        1411 non-null   int64         
 1   date              1411 non-null   datetime64[ns]
 2   S199              1411 non-null   float64       
 3   S200              1411 non-null   float64       
 4   S332B             1411 non-null   float64       
 5   S332BN            1411 non-null   float64       
 6   S332C             1411 non-null   float64       
 7   S332D             1411 non-null   float64       
 8   S332DX1           1411 non-null   float64       
 9   S12A              1411 non-null   int64         
 10  S12B              1411 non-null   int64         
 11  S12C              1411 non-null   int64         
 12  S12D              1411 non-null   int64         
 13  S333              1411 non-null   float64       
 14  S333N             1411 n

In [30]:
# Drop the 'formatted_date' column if not needed
data = data.drop(columns=['Unnamed: 0','formatted_date'])

data['date'] = data['date'].dt.strftime('%m/%d/%Y')

# Check the data to confirm the index
data

,date,S199,S200,S332B,S332BN,S332C,S332D,S332DX1,S12A,S12B,...,NP205_rain,NP205_stage,P33_rain,P33_stage,NP205_PET,P33_PET,G620_water_level,SWEVER4_stage,TSH_stage,NP62_stage
0,10/16/2020,242.17,279.69,218.42,218.42,445.77,481.38,0.0,460,382,...,2.28,6.925,0.106,7.133,2.09,2.09,5.92,2.92,3.03,4.04
1,10/17/2020,296.05,294.85,265.13,265.13,587.35,556.24,0.0,490,405,...,0.01,7.025,0.000,7.133,3.87,3.87,5.97,2.95,3.08,4.05
2,10/18/2020,277.85,294.24,264.23,264.23,514.45,503.14,0.0,472,394,...,0.00,7.008,0.088,7.138,3.51,3.51,5.98,2.93,3.06,4.05
3,10/19/2020,224.13,294.72,264.39,264.39,491.06,537.91,0.0,466,392,...,0.11,6.986,0.256,7.167,1.98,1.98,5.98,2.91,3.05,4.06
4,10/20/2020,260.51,295.06,263.38,263.38,489.25,558.77,0.0,472,399,...,0.37,6.977,0.678,7.195,1.27,1.27,5.99,2.90,3.06,4.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1406,08/22/2024,297.72,294.23,0.17,0.17,30.02,559.39,0.0,358,250,...,1.72,7.289,0.050,7.466,2.75,2.75,6.13,2.85,2.78,3.88
1407,08/23/2024,240.47,293.74,0.45,0.45,0.29,491.92,0.0,375,265,...,0.04,7.360,0.000,7.447,2.75,2.75,6.18,2.90,2.81,3.88
1408,08/24/2024,242.45,293.75,0.00,0.00,0.00,439.40,0.0,377,268,...,0.00,7.360,0.010,7.435,2.75,2.75,6.20,2.88,2.80,3.88
1409,08/25/2024,286.80,294.24,0.00,0.00,0.00,470.49,0.0,380,272,...,0.00,7.345,0.270,7.440,2.75,2.75,6.19,2.86,2.80,3.98


In [31]:
# load pretrain model
#model = AutoModelForCausalLM.from_pretrained('thuml/timer-base-84m', trust_remote_code=True)

In [32]:
# perpare input
#lookback_length = 100      # support the maximum context length up to 2880 
#seqs = torch.tensor(data["G620_water_level"][:lookback_length]).unsqueeze(0).float()

In [33]:
#seqs

In [34]:
# generate forecast
#prediction_length = 7
#output = model.generate(seqs, max_new_tokens=prediction_length)

#print(output.shape)

In [35]:
#output

In [36]:
# Convert 'date' column to datetime type
data['date'] = pd.to_datetime(data['date'])

# Now you can safely split the data
val_size = int(0.15 * len(data['date'].unique()))
test_size = int(0.15 * len(data['date'].unique()))

# Split the data based on the datetime values
Y_train_df = data[data['date'] < data['date'].values[-val_size - test_size]]  # Training set
Y_val_df = data[(data['date'] >= data['date'].values[-val_size - test_size]) & (data['date'] < data['date'].values[-test_size])]  # Validation set
Y_test_df = data[data['date'] >= data['date'].values[-test_size]].reset_index(drop=True)  # Test set

In [37]:
Y_train_df

,date,S199,S200,S332B,S332BN,S332C,S332D,S332DX1,S12A,S12B,...,NP205_rain,NP205_stage,P33_rain,P33_stage,NP205_PET,P33_PET,G620_water_level,SWEVER4_stage,TSH_stage,NP62_stage
0,2020-10-16,242.17,279.69,218.42,218.42,445.77,481.38,0.00,460,382,...,2.28,6.925,0.106,7.133,2.09,2.09,5.92,2.92,3.03,4.04
1,2020-10-17,296.05,294.85,265.13,265.13,587.35,556.24,0.00,490,405,...,0.01,7.025,0.000,7.133,3.87,3.87,5.97,2.95,3.08,4.05
2,2020-10-18,277.85,294.24,264.23,264.23,514.45,503.14,0.00,472,394,...,0.00,7.008,0.088,7.138,3.51,3.51,5.98,2.93,3.06,4.05
3,2020-10-19,224.13,294.72,264.39,264.39,491.06,537.91,0.00,466,392,...,0.11,6.986,0.256,7.167,1.98,1.98,5.98,2.91,3.05,4.06
4,2020-10-20,260.51,295.06,263.38,263.38,489.25,558.77,0.00,472,399,...,0.37,6.977,0.678,7.195,1.27,1.27,5.99,2.90,3.06,4.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
984,2023-06-27,0.00,0.00,323.13,264.45,164.83,366.83,109.61,0,0,...,0.00,6.705,0.000,6.971,5.27,5.27,5.59,2.74,2.77,3.51
985,2023-06-28,0.00,0.00,265.67,265.66,115.36,396.88,135.67,0,0,...,0.02,6.660,0.040,6.962,5.31,5.31,5.56,2.71,2.76,3.52
986,2023-06-29,0.00,0.00,185.67,185.38,120.09,438.17,192.19,0,0,...,0.01,6.611,0.020,6.960,4.93,4.93,5.55,2.69,2.75,3.52
987,2023-06-30,0.00,0.00,266.31,223.21,210.21,439.50,192.33,0,0,...,0.05,6.571,0.470,6.980,3.44,3.44,5.53,2.73,2.79,3.53


In [38]:
Y_test_df

,date,S199,S200,S332B,S332BN,S332C,S332D,S332DX1,S12A,S12B,...,NP205_rain,NP205_stage,P33_rain,P33_stage,NP205_PET,P33_PET,G620_water_level,SWEVER4_stage,TSH_stage,NP62_stage
0,2024-01-29,77.29,152.88,0.00000,0.00,0.00,177.14,0.0,245,197,...,0.00,7.127,0.00,7.277,4.09,4.09,5.99,2.70,2.70,3.69
1,2024-01-30,76.12,152.88,0.00000,0.00,0.00,234.49,0.0,245,194,...,0.00,7.116,0.00,7.262,3.72,3.72,5.97,2.68,2.69,3.68
2,2024-01-31,76.11,152.88,0.00000,0.00,0.00,140.48,0.0,241,191,...,0.00,7.105,0.00,7.253,3.98,3.98,5.96,2.66,2.67,3.67
3,2024-02-01,76.04,151.21,2.12873,0.00,0.00,150.86,0.0,240,189,...,0.00,7.096,0.00,7.242,3.14,3.14,5.95,2.64,2.66,3.65
4,2024-02-02,76.01,150.44,4.25746,0.00,0.00,249.52,0.0,236,188,...,0.00,7.090,0.00,7.236,3.40,3.40,5.94,2.63,2.63,3.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,2024-08-22,297.72,294.23,0.17000,0.17,30.02,559.39,0.0,358,250,...,1.72,7.289,0.05,7.466,2.75,2.75,6.13,2.85,2.78,3.88
207,2024-08-23,240.47,293.74,0.45000,0.45,0.29,491.92,0.0,375,265,...,0.04,7.360,0.00,7.447,2.75,2.75,6.18,2.90,2.81,3.88
208,2024-08-24,242.45,293.75,0.00000,0.00,0.00,439.40,0.0,377,268,...,0.00,7.360,0.01,7.435,2.75,2.75,6.20,2.88,2.80,3.88
209,2024-08-25,286.80,294.24,0.00000,0.00,0.00,470.49,0.0,380,272,...,0.00,7.345,0.27,7.440,2.75,2.75,6.19,2.86,2.80,3.98


In [39]:
import torch
torch.set_float32_matmul_precision('medium')

In [40]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Use only GPU

In [41]:
# load pretrain model
model = AutoModelForCausalLM.from_pretrained('thuml/timer-base-84m', trust_remote_code=True)

/aul/homes/rrang016/.conda/envs/py310/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [42]:
# Set the prediction start date
#prediction_start_date = '2024-01-29'

# Filter training data up to 01/28/2024
#train_data = data[data['date'] < prediction_start_date]
#train_data

In [43]:
# Prepare input
#lookback_length = 100      # Support the maximum context length up to 2880 
#seqs = torch.tensor(train_data["G620_water_level"].tail(lookback_length).values).unsqueeze(0).float()

In [44]:
#seqs

In [45]:
# generate forecast
#prediction_length = 7
#output = model.generate(seqs, max_new_tokens=prediction_length)

#print(output.shape)
#print(output)

In [46]:
### 28days ####

# Initialize an empty list to store predictions
predictions_list = []

# Loop over the date range for rolling window (up to a certain date)
for current_date in pd.date_range('01-01-2024', '2024-07-29', freq='D'):
    print(f"Processing date: {current_date}")  # Monitor progress

    # Filter the data to include only the last 100 days up to and including current_date
    train_data_window = data[data['date'] <= current_date]  # Filter rows up to the current date
    last_100_days_data = train_data_window.tail(100)  # Get the last 100 days

    # Loop over each column (variable) for the wide format dataset
    for variable in last_100_days_data.columns:
        if variable in ['date']:  # Skip the non-time series columns like 'date'
            continue

        print(f"  Processing variable: {variable},  ds: {current_date}")  # Monitor progress
        
        # Prepare input for the Timer model (last 'lookback_length' data)
        lookback_length = 100  # Use the last 100 days for forecasting
        seqs = torch.tensor(last_100_days_data[variable].values).unsqueeze(0).float()

        # Generate forecast (predict the next 28 days)
        prediction_length = 28
        output = model.generate(seqs, max_new_tokens=prediction_length)

        # Extract forecasted values (all 28 days)
        forecast_values = output.squeeze().detach().numpy()

        # Append the 28th day prediction to predictions list (the last forecast)
        forecast_28th_day = forecast_values[-1]  # Get the 28th day forecast

        # Append the 28th day forecast value to the list
        predictions_list.append({
            'variable': variable,
            'ds': current_date + pd.Timedelta(days=28),  # Forecast date for the 28th day
            'forecast_value': forecast_28th_day
        })

# Convert predictions list to DataFrame for better visualization
predictions_df = pd.DataFrame(predictions_list)

# Print some predictions to check
print(predictions_df.head())


Processing date: 2024-01-01 00:00:00
  Processing variable: S199,  ds: 2024-01-01 00:00:00
  Processing variable: S200,  ds: 2024-01-01 00:00:00
  Processing variable: S332B,  ds: 2024-01-01 00:00:00
  Processing variable: S332BN,  ds: 2024-01-01 00:00:00
  Processing variable: S332C,  ds: 2024-01-01 00:00:00
  Processing variable: S332D,  ds: 2024-01-01 00:00:00
  Processing variable: S332DX1,  ds: 2024-01-01 00:00:00
  Processing variable: S12A,  ds: 2024-01-01 00:00:00
  Processing variable: S12B,  ds: 2024-01-01 00:00:00
  Processing variable: S12C,  ds: 2024-01-01 00:00:00
  Processing variable: S12D,  ds: 2024-01-01 00:00:00
  Processing variable: S333,  ds: 2024-01-01 00:00:00
  Processing variable: S333N,  ds: 2024-01-01 00:00:00
  Processing variable: S334,  ds: 2024-01-01 00:00:00
  Processing variable: S343A,  ds: 2024-01-01 00:00:00
  Processing variable: S343B,  ds: 2024-01-01 00:00:00
  Processing variable: S344,  ds: 2024-01-01 00:00:00
  Processing variable: S356,  ds: 

In [47]:
predictions_df

,variable,ds,forecast_value
0,S199,2024-01-29,196.997757
1,S200,2024-01-29,219.735626
2,S332B,2024-01-29,279.844452
3,S332BN,2024-01-29,150.332504
4,S332C,2024-01-29,260.200043
...,...,...,...
7802,P33_PET,2024-08-26,2.901345
7803,G620_water_level,2024-08-26,5.695037
7804,SWEVER4_stage,2024-08-26,2.524436
7805,TSH_stage,2024-08-26,2.546829


In [48]:
# Sort the DataFrame by unique_id and ds to group the data properly
predictions_df1 = predictions_df.sort_values(by=['variable', 'ds']).reset_index(drop=True)
predictions_df1

,variable,ds,forecast_value
0,G620_water_level,2024-01-29,6.309862
1,G620_water_level,2024-01-30,6.306299
2,G620_water_level,2024-01-31,6.299726
3,G620_water_level,2024-02-01,6.283675
4,G620_water_level,2024-02-02,6.279122
...,...,...,...
7802,TSH_stage,2024-08-22,2.409807
7803,TSH_stage,2024-08-23,2.433961
7804,TSH_stage,2024-08-24,2.441401
7805,TSH_stage,2024-08-25,2.461115


In [49]:
Y_test_df

,date,S199,S200,S332B,S332BN,S332C,S332D,S332DX1,S12A,S12B,...,NP205_rain,NP205_stage,P33_rain,P33_stage,NP205_PET,P33_PET,G620_water_level,SWEVER4_stage,TSH_stage,NP62_stage
0,2024-01-29,77.29,152.88,0.00000,0.00,0.00,177.14,0.0,245,197,...,0.00,7.127,0.00,7.277,4.09,4.09,5.99,2.70,2.70,3.69
1,2024-01-30,76.12,152.88,0.00000,0.00,0.00,234.49,0.0,245,194,...,0.00,7.116,0.00,7.262,3.72,3.72,5.97,2.68,2.69,3.68
2,2024-01-31,76.11,152.88,0.00000,0.00,0.00,140.48,0.0,241,191,...,0.00,7.105,0.00,7.253,3.98,3.98,5.96,2.66,2.67,3.67
3,2024-02-01,76.04,151.21,2.12873,0.00,0.00,150.86,0.0,240,189,...,0.00,7.096,0.00,7.242,3.14,3.14,5.95,2.64,2.66,3.65
4,2024-02-02,76.01,150.44,4.25746,0.00,0.00,249.52,0.0,236,188,...,0.00,7.090,0.00,7.236,3.40,3.40,5.94,2.63,2.63,3.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,2024-08-22,297.72,294.23,0.17000,0.17,30.02,559.39,0.0,358,250,...,1.72,7.289,0.05,7.466,2.75,2.75,6.13,2.85,2.78,3.88
207,2024-08-23,240.47,293.74,0.45000,0.45,0.29,491.92,0.0,375,265,...,0.04,7.360,0.00,7.447,2.75,2.75,6.18,2.90,2.81,3.88
208,2024-08-24,242.45,293.75,0.00000,0.00,0.00,439.40,0.0,377,268,...,0.00,7.360,0.01,7.435,2.75,2.75,6.20,2.88,2.80,3.88
209,2024-08-25,286.80,294.24,0.00000,0.00,0.00,470.49,0.0,380,272,...,0.00,7.345,0.27,7.440,2.75,2.75,6.19,2.86,2.80,3.98


In [50]:
#Convert 'date' to datetime format if not 
Y_test_df['date'] = pd.to_datetime(Y_test_df['date'])

#Melt the DataFrame to long format
long_df = pd.melt(Y_test_df, id_vars=['date'], 
                  value_vars=[col for col in Y_test_df.columns if col != 'date'], 
                  var_name='unique_id', 
                  value_name='y')

#Rename the date column to 'ds'
long_df.rename(columns={'date': 'ds'}, inplace=True)

# Display the resulting long format DataFrame
print(long_df) 

             ds   unique_id      y
0    2024-01-29        S199  77.29
1    2024-01-30        S199  76.12
2    2024-01-31        S199  76.11
3    2024-02-01        S199  76.04
4    2024-02-02        S199  76.01
...         ...         ...    ...
7802 2024-08-22  NP62_stage   3.88
7803 2024-08-23  NP62_stage   3.88
7804 2024-08-24  NP62_stage   3.88
7805 2024-08-25  NP62_stage   3.98
7806 2024-08-26  NP62_stage   4.03

[7807 rows x 3 columns]


In [51]:
predictions_df1 = predictions_df1.rename(columns={'variable': 'unique_id', 'forecast_value':'Timer'})
predictions_df1

,unique_id,ds,Timer
0,G620_water_level,2024-01-29,6.309862
1,G620_water_level,2024-01-30,6.306299
2,G620_water_level,2024-01-31,6.299726
3,G620_water_level,2024-02-01,6.283675
4,G620_water_level,2024-02-02,6.279122
...,...,...,...
7802,TSH_stage,2024-08-22,2.409807
7803,TSH_stage,2024-08-23,2.433961
7804,TSH_stage,2024-08-24,2.441401
7805,TSH_stage,2024-08-25,2.461115


In [52]:
results_df = pd.merge(predictions_df1, long_df, on=['unique_id', 'ds'], how='inner')
results_df

,unique_id,ds,Timer,y
0,G620_water_level,2024-01-29,6.309862,5.99
1,G620_water_level,2024-01-30,6.306299,5.97
2,G620_water_level,2024-01-31,6.299726,5.96
3,G620_water_level,2024-02-01,6.283675,5.95
4,G620_water_level,2024-02-02,6.279122,5.94
...,...,...,...,...
7802,TSH_stage,2024-08-22,2.409807,2.78
7803,TSH_stage,2024-08-23,2.433961,2.81
7804,TSH_stage,2024-08-24,2.441401,2.80
7805,TSH_stage,2024-08-25,2.461115,2.80


In [53]:
import numpy as np
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse

# List of stages to evaluate
stages = ['NP205_stage', 'P33_stage','G620_water_level', 'NESRS1', 'NESRS2']

for stage in stages:
    # Filter the data for the specific stage
    stage_data = results_df[results_df['unique_id'] == stage]
    print(f"======== {stage} ========")
    
    # Extract the actual and predicted values
    true = stage_data['y'].values
    pred = stage_data['Timer'].values
    
    # Calculate MAE and RMSE
    mae_value = mae(true, pred)
    rmse_value = np.sqrt(mse(true, pred))
    
    # Print the results
    print(f'MAE = {mae_value:.3f}')
    print(f'RMSE = {rmse_value:.3f}\n')

======== NP205_stage ========
MAE = 0.611
RMSE = 0.784

======== P33_stage ========
MAE = 0.322
RMSE = 0.366

======== G620_water_level ========
MAE = 0.446
RMSE = 0.501

======== NESRS1 ========
MAE = 0.294
RMSE = 0.331

======== NESRS2 ========
MAE = 0.251
RMSE = 0.291



In [54]:
import numpy as np
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse

# List of stages to evaluate
stages = ['NP205_stage', 'P33_stage', 'G620_water_level', 'NESRS1', 'NESRS2']

# List of models (these correspond to column names in the DataFrame)
models = ['Timer']

# Dictionary to store overall MAE and RMSE for each model
overall_mae = {model: [] for model in models}
overall_rmse = {model: [] for model in models}

# Iterate through each stage
for stage in stages:
    # Filter the data for the specific stage
    stage_data = results_df[results_df['unique_id'] == stage]
    
    # Check if data for the stage exists
    if stage_data.empty:
        print(f"No data found for {stage}. Skipping...\n")
        continue
    
    # Extract the actual values
    true = stage_data['y'].values
    
    # Iterate through each model
    for model in models:
        # Extract the predicted values for the model
        pred = stage_data[model].values
        
        # Calculate MAE and RMSE, then append to the corresponding model's list
        model_mae = mae(true, pred)
        model_rmse = np.sqrt(mse(true, pred))
        
        overall_mae[model].append(model_mae)
        overall_rmse[model].append(model_rmse)

# Calculate the overall MAE and RMSE (mean of MAEs and RMSEs across the 5 stations) for each model
final_overall_mae = {model: np.mean(maes) for model, maes in overall_mae.items()}
final_overall_rmse = {model: np.mean(rmses) for model, rmses in overall_rmse.items()}

# Print the overall MAE and RMSE for each model
print("Overall MAE for Timer (only for selected stages):")
for model, mae_value in final_overall_mae.items():
    print(f"{model}: MAE = {mae_value:.3f}")

print("\nOverall RMSE for Timer (only for selected stages):")
for model, rmse_value in final_overall_rmse.items():
    print(f"{model}: RMSE = {rmse_value:.3f}")

Overall MAE for Timer (only for selected stages):
Timer: MAE = 0.385

Overall RMSE for Timer (only for selected stages):
Timer: RMSE = 0.455


In [ ]:
# Save predictions to a CSV file
results_df.to_csv('Output/Timer-Predictions-28days.csv', index=False)